In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error


host = '127.0.0.1'
user = 'Felipe'
password = 'JUULisCUUL420!'  
database = 'SchoolDB'


csv_file_path = '/Users/felipequiroz/Desktop/DS5110/DataProject/responses.csv'


In [2]:
try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    if connection.is_connected():
        print("Connected to MySQL database")
        cursor = connection.cursor()
        df = pd.read_csv(csv_file_path)
        print("CSV file loaded successfully")
        df = df.where(pd.notnull(df), None)
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Students (student_id, name, email)
                VALUES (%s, %s, %s)
                ON DUPLICATE KEY UPDATE name=VALUES(name), email=VALUES(email)
            """, (row['student_id'], row['name'], row['email']))
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Classes (class_id, class_name, credits, department)
                VALUES (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE class_name=VALUES(class_name), credits=VALUES(credits), department=VALUES(department)
            """, (row['class_id'], row['class_name'], row['credits'], row['department']))
        for _, row in df.iterrows():
            cursor.execute("""
                INSERT INTO Student_Classes (student_id, class_id, can_tutor, tutoring_availability)
                VALUES (%s, %s, %s, %s)
            """, (
                row['student_id'], row['class_id'],
                row['can_tutor'] == "Yes", 
                row['tutoring_availability']
            ))
        for _, row in df.iterrows():
            if not pd.isnull(row['note_id']): 
                cursor.execute("""
                    INSERT INTO Course_Notes (note_id, class_id, note_title, content)
                    VALUES (%s, %s, %s, %s)
                """, (row['note_id'], row['class_id'], row['note_title'], row['content']))\
        for _, row in df.iterrows():
            if not pd.isnull(row['notebook_id']):  
                cursor.execute("""
                    INSERT INTO Notebooks (notebook_id, class_id, title, file)
                    VALUES (%s, %s, %s, %s)
                """, (row['notebook_id'], row['class_id'], row['title'], row['file']))
        connection.commit()
        print("All data inserted successfully into the database")

except Error as e:
    print(f"Error: {e}")

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")

Connected to MySQL database
CSV file loaded successfully
All data inserted successfully into the database
MySQL connection closed
